# Text generation using Mixture of Experts

*Copyright (c) 2023 Graphcore Ltd.*

This notebook outlines a character-level text generation model in PyTorch, showing how, starting from a standard model definition, we can define a **mixture of experts** model. The notebook is written in PyTorch, and allows for the model to be run on both CPU and IPUs.

## Installation

Let's start with installing the required modules:

In [ ]:
%pip install -q numpy torch matplotlib
%pip install -q git+https://github.com/graphcore-research/poptorch-experimental-addons@14886d2285c3e45b0eadf4d719dae87d5f28b109

We will also download the dataset and model checkpoints:

In [ ]:
%%bash
wget -q -P /tmp/ https://graphcore-research-public.s3.eu-west-1.amazonaws.com/2023-moe-notebook/wikitext103.zip
unzip /tmp/wikitext103.zip -d /tmp/dataset_cache/
rm /tmp/wikitext103.zip
wget -q -P /tmp/ https://graphcore-research-public.s3.eu-west-1.amazonaws.com/2023-moe-notebook/checkpoints.zip
unzip /tmp/checkpoints.zip -d /tmp/
rm /tmp/checkpoints.zip

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import poptorch
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Tuple, Optional

In [ ]:
dataset_dir = Path(os.getenv("DATASET_DIR"))
checkpoint_dir = Path(os.getenv("CHECKPOINT_DIR"))

## Dataset

For training, we use the freely-available character-level `WikiText-103` dataset that can be downloaded from [here](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/). In order to load and batch the data, we can use `Dataset` class from `data.py`. The class requires a corresponding character-level vocabulary that we will use to convert between the characters and the corresponding integer indices. In this case, the vocabulary is pre-defined in `vocab.json` and it contains all 5,008 different characters appearing in `WikiText-103`. Internally, `Dataset` class uses the `CharVocab` class from `data.py` to provide the `str_to_ids` and `ids_to_str` methods for converting between text and an array of character ids:

In [ ]:
from data import CharVocab

vocab = CharVocab.from_path(dataset_dir / "wikitext103_raw/vocab.json")
print(f"Number of characters in the vocabulary: {len(vocab):,}")

some_text = "Hello world"
char_ids = vocab.str_to_ids(some_text)

print(f"{some_text} -> {char_ids} -> {vocab.ids_to_str(char_ids)}")

When defining a `Dataset` object, we need to pass the text and the vocabulary (which will internally be converted to a `CharVocab` object). Let's load up the `validation` dataset to see how it works (it is smaller than the training data so will load more quickly).

**Note**: When the dataset is loaded, the whole text is processed and converted to integer ids - this may take a bit of time.

#### Load the dataset

In [ ]:
from data import Dataset

vocab_path = dataset_dir / "wikitext103_raw/vocab.json"
text_path = dataset_dir / "wikitext103_raw/valid.txt"

ds = Dataset.from_path(vocab_path, text_path)

print(f"Number of characters in the dataset: {len(ds):,}")

#### Batching the data

Now that we have our `Dataset` object, we can call the `batch()` method in order to return an iterator that will sequentially generate batches of our training, validation, or test data. For the `batch` method, we need to pass the `batch_size` (number of sequences in a batch), `sequence_length` (number of characters per sequence), as well as `overlap_length` (number of characters that overlap between consecutive sequences). Note that this means that start indices of consecutive sequences will be `sequence_length - overlap_length` characters apart. In addition, we can pass a `seed` parameter to randomly generate the order of batched sequences - leaving it to `None` will instead generate sequences in the order of appearance within the dataset.

Each batch will consist of an input/target pair of character id sequences (where for each input, the target is shifted one place to the left, i.e. corresponding to the next character in the sequence), as well as a mask tensor that masks out the first `overlap_length` number of characters in each sequence - we will use this during training in order to prevent the first `overlap_length` character predictions to contribute to the model loss.

In [ ]:
batch_size = 8
sequence_length = 256
overlap_length = 32
seed = 100

batch = next(
    ds.batch(
        batch_size,
        sequence_length,
        overlap_length,
        seed,
    )
)
x = batch["x"]
y = batch["y"]
mask = batch["mask"]

print(f"Shape of the input: {x.shape}")
print(f"Shape of the output: {y.shape}")

seq_num = 1
print("Example input sequence:\n", ds.vocab.ids_to_str(x[seq_num]))
print("Example target sequence:\n", ds.vocab.ids_to_str(y[seq_num]))
print("Sequence mask:\n", mask[seq_num])

## Baseline model

Let's start by defining the character-level text-generation baseline model. In this example we will use a simplified transformer-style architecture where instead of a multi-head attention block we use a causal one-dimensional convolutional layer. We construct the model by alternating convolutional and FFN layers.

#### Convolutional layer

In order to construct a *causal* convolution, we use a regular `nn.Conv1d` layer with `in_channels=hidden_size` and `out_channels=hidden_size`, with the setting `padding=kernel_size - 1` which ensures that the output at each position in the sequence only depends on the *previous* characters in the input sequence. As this leads to additional dummy elements at the end of each sequence, we cut the last `kernel_size - 1` characters in the `forward` method. Note that additional reshaping and axis permutations need to be done in the `forward` method to align with the expected `Conv1d` input order.

In [ ]:
class CausalConv1d(nn.Conv1d):
    """Conv1d layer with left padding to enforce causality"""

    def __init__(
        self,
        hidden_size: int,
        kernel_size: int,
        num_groups: int,
        dtype: torch.dtype,
    ):
        super().__init__(
            in_channels=hidden_size,
            out_channels=hidden_size,
            kernel_size=kernel_size,
            padding=kernel_size - 1,
            groups=num_groups,
            dtype=dtype,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.transpose(-1, -2)
        x = super().forward(x)[..., : -(self.kernel_size[0] - 1)]
        return x.transpose(-1, -2)

#### FFN layer

For the FFN layer, we will use the standard transformer structure of an up and down projection with a nonlinearity in between. For this it's useful to define helper functions for initialising the layer weights and biases tensors.

In [ ]:
def create_weight_tensor(shape: Tuple[int, ...], dtype: torch.dtype) -> nn.Parameter:
    w = nn.Parameter(torch.empty(shape, dtype=dtype))
    xavier_std = np.sqrt(2 / (shape[-2] + shape[-1]))
    return nn.init.normal_(w, mean=0, std=xavier_std)


def create_bias_tensor(out_features: int, dtype: torch.dtype) -> nn.Parameter:
    b = nn.Parameter(torch.empty(out_features, dtype=dtype))
    return nn.init.zeros_(b)

In [ ]:
class FFN(nn.Module):
    """Transformer-style FFN layer"""

    def __init__(self, hidden_size: int, expand_factor: int, dtype: torch.dtype):
        super().__init__()
        self.hidden_size, self.expand_factor, self.dtype = (
            hidden_size,
            expand_factor,
            dtype,
        )
        self.w1 = create_weight_tensor(
            (hidden_size, hidden_size * expand_factor), dtype
        )
        self.w2 = create_weight_tensor(
            (hidden_size * expand_factor, hidden_size), dtype
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return F.relu(x @ self.w1) @ self.w2

#### Putting it all together

Now that we have defined our convolutional and FFN layers, we can build the core layer of our model following the Transformer recipe: sequence of `CausalConv1d` and `FFN` layers with residual connections and layer normalizations between.

In [ ]:
class CoreLayer(nn.Module):
    """Core model layer consisting of Conv1D + normalization + FFN"""

    def __init__(
        self,
        hidden_size: int,
        expand_factor: int,
        kernel_size: int,
        num_groups: int,
        dtype: torch.dtype,
    ):
        super().__init__()

        self.ffn = FFN(
            hidden_size=hidden_size, expand_factor=expand_factor, dtype=dtype
        )
        self.conv = CausalConv1d(
            hidden_size=hidden_size,
            kernel_size=kernel_size,
            num_groups=num_groups,
            dtype=dtype,
        )
        self.layer_norm1 = nn.LayerNorm(hidden_size, dtype=dtype)
        self.layer_norm2 = nn.LayerNorm(hidden_size, dtype=dtype)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_conv = F.relu(self.conv(x))
        x = self.layer_norm1(x_conv + x)
        x_ffn = self.ffn(x)
        return self.layer_norm2(x_ffn + x)

The full model will now consist of a sequence of `CoreLayer` layers, with an additional `nn.Embedding` for the input embeddings, and a linear output projection from `hidden_size` to `vocab_size`. We additionally define the `masked_cross_entropy_loss` method in order to ignore the loss contributions of the characters based on the previously generated `mask`.

In order to ensure our model can run on both IPUs and CPU we need to calculate and return the loss function within our `forward` method, as well as to wrap it within `poptorch.identity_loss` function.

In [ ]:
class CharModel(nn.Module):
    """Character-level language model"""

    def __init__(
        self,
        vocab_size: int,
        hidden_size: int,
        expand_factor: int,
        kernel_size: int,
        num_groups: int,
        num_layers: int,
        dtype: torch.dtype,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, hidden_size, dtype=dtype)
        self.core_layers = nn.ModuleList()
        for _ in range(num_layers):
            self.core_layers.append(
                CoreLayer(
                    hidden_size=hidden_size,
                    expand_factor=expand_factor,
                    kernel_size=kernel_size,
                    num_groups=num_groups,
                    dtype=dtype,
                )
            )
        self.w_out = create_weight_tensor(shape=(hidden_size, vocab_size), dtype=dtype)
        self.b_out = create_bias_tensor(out_features=vocab_size, dtype=dtype)

    def masked_cross_entropy_loss(
        self, logits: torch.Tensor, y: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        batch_loss = F.cross_entropy(
            logits.reshape((-1, self.vocab_size)),
            y.reshape(-1),
            reduction="none",
        )
        return (batch_loss * mask.reshape(-1)).sum() / mask.sum()

    def forward(
        self, x: torch.Tensor, mask: torch.Tensor = None, y: torch.Tensor = None
    ) -> torch.Tensor:
        embed = self.embedding(x)
        for layer in self.core_layers:
            embed = layer(embed)
        logits = embed @ self.w_out + self.b_out

        if self.training:
            loss = self.masked_cross_entropy_loss(logits, y, mask)
            return logits, poptorch.identity_loss(loss, reduction="none")

        return logits

#### Training the model *(can be skipped)*



Now that we have defined all modules of our model, we can build and train the model on the `WikiText-103` dataset.

For training, we use the `AdamW` optimiser with learning rate decay. The following code showcases a basic training loop with these settings, making sure that the model is wrapped within `poptorch.trainingModel` when executing on the IPU. Note that running on the CPU is very slow and is mainly provided as a reference.

In [ ]:
# # Load the training dataset

# vocab_path = dataset_dir / "wikitext103_raw/vocab.json"
# train_path = dataset_dir / "wikitext103_raw/train.txt"

# train_ds = Dataset.from_path(vocab_path, train_path)

In [ ]:
# # Set random seed
# seed = None
# if seed is None:
#     seed = np.random.SeedSequence().generate_state(1)[0]

# # Model hyperparameters
# model = CharModel(
#     vocab_size=len(ds.vocab),
#     hidden_size=128,
#     expand_factor=4,
#     kernel_size=7,
#     num_groups=8,
#     num_layers=8,
#     dtype=torch.float32,
# )

# # Batch settings
# batch_size = 32
# sequence_length = 256
# overlap_length = 32
# batch_seed = 100


# # Change this to switch between IPU and CPU execution
# run_on_ipu = True
# num_replicas = 4

# # Training parameters
# lr = 2e-3
# lr_decay = 2**-16
# num_steps = 2**15


# # Set the optimizer
# if run_on_ipu:
#     opts = poptorch.Options()
#     opts.replicationFactor(num_replicas)
#     optimizer = poptorch.optim.AdamW(model.parameters(), lr=lr)
#     poptorch_model = poptorch.trainingModel(model, opts, optimizer)
# else:
#     optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


# # Training loop
# model.train()
# lr_decay_factor = 2**-lr_decay
# training_loss = {}
# for step, batch in zip(
#     range(num_steps),
#     train_ds.batch(batch_size, sequence_length, overlap_length, batch_seed),
# ):
#     # Apply a step
#     if run_on_ipu:
#         out, loss = poptorch_model(batch["x"].int(), batch["mask"], batch["y"].int())
#         loss = torch.mean(loss)  # average over IPUs if num_replicas > 1
#     else:
#         optimizer.zero_grad()
#         out, loss = model(batch["x"], batch["mask"], batch["y"])
#         loss.backward()
#         optimizer.step()

#     training_loss[step] = loss

#     # Print results
#     if step % 2**8 == 0:
#         print(f"Step {step}:")
#         print(f"Loss: {loss}")
#         print()

#     # Learning rate schedule
#     lr *= lr_decay_factor
#     optimizer.param_groups[0]["lr"] = lr
#     if run_on_ipu:
#         poptorch_model.setOptimizer(optimizer)

In [ ]:
# # Plot training loss

# import matplotlib.pyplot as plt

# plt.plot(training_loss.values())
# plt.xlabel("Step")
# plt.ylabel("Cross entropy loss")
# plt.title("Training loss")
# plt.show()

#### Generating text

Let's see our baseline model in action! We can load a checkpoint of the model and use it in inference (or alternatively use the model we trained in the previous section).

In [ ]:
def sample_text(
    model: CharModel, text: str, num_characters: int, temperature: float = 1.0
) -> str:
    print(text, end="")
    model.train(False)
    ids = vocab.str_to_ids(text)

    for _ in range(num_characters):
        y = model(ids)
        new_char = torch.multinomial(
            F.softmax(y[-1] / temperature, dim=0), num_samples=1
        )
        ids = torch.cat((ids, torch.tensor([new_char])), dim=0)
        print(vocab.ids_to_str(new_char), end="", flush=True)
    print()
    return vocab.ids_to_str(ids)

In [ ]:
model_from_checkpoint = CharModel(
    vocab_size=len(ds.vocab),
    hidden_size=128,
    expand_factor=4,
    kernel_size=7,
    num_groups=8,
    num_layers=8,
    dtype=torch.float32,
)
model_from_checkpoint.load_state_dict(torch.load(checkpoint_dir / "dense.pt"))

out = sample_text(
    model_from_checkpoint, "Hello worl", num_characters=300, temperature=0.4
)

Or we can sample from the model we trained in the notebook:

In [ ]:
# # Sample from trained model
# print(sample_text(model, "Hello world", num_characters=300, temperature=0.5))

## Mixture of experts

We will now see how we can turn our regular *dense* model into a *mixture of experts* model that we can run on IPUs/CPU.

### Overview

In the standard mixture of experts setup, we take some of the FFN layers of the model and turn them into sparse MoE layers. This is done by instantiating multiple copies of the basic FFN layer with independent parameters (i.e. experts), and routing the inputs to different experts based on a trained scoring function. For the purposes of this notebook, we will demonstrate how to define a setup closely following the architecture presented in the [Switch Transformers](https://arxiv.org/abs/2101.03961) paper.

### Switch router

Let's first tackle the problem of assigning individual tokens (throughout this notebook, characters) to experts, assuming we have a batch of tokens of shape `(batch_size, hidden_size)` that we want to allocate to `num_experts` experts. The starting point is to calculate a score for each token-expert assignment pair, and use these scores to assign tokens to experts. The optimal assignment problem can therefore be stated as maximising the total score of the assignments, while keeping the constraint that each expert is assigned exactly `expert_capacity = batch_size / num_experts` tokens. As solving this optimisation problem is computationally expensive, the Switch Transformer uses a simplified approach: assign each token to its preferred expert based on the scores, and afterwards make sure that each expert receives the same number of tokens. This is done by randomly disregarding some tokens for the over-assigned experts, and padding with "dummy" tokens the inputs to under-assigned experts.

In order to train the scoring function throughout training, when the token `i` is routed to the expert `j`, the output of the expert is multiplied by `score[i, j]` which tends to strengthen the "successful" token-expert assignments and weaken the "unsuccessful" ones.

#### Scoring function

For the scoring function, we will use a simple linear feed-forward layer producing the output of shape `(batch_size, num_experts)` and normalising so that the scores for each token sum to 1.

In [ ]:
class Scorer(nn.Module):
    def __init__(self, hidden_size: int, num_experts: int, dtype: torch.dtype):
        super().__init__()
        self.w = create_weight_tensor((hidden_size, num_experts), dtype)
        self.b = create_bias_tensor(num_experts, dtype)
        self.dtype = dtype

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Return per-token expert scores (normalised so rows sum to 1)"""
        assert len(x.shape) == 2

        logits = x @ self.w + self.b
        return F.softmax(logits, dim=1, dtype=self.dtype)

#### Assigning tokens to experts

The output of the `Scorer` module is the scores matrix of shape `(batch_size, num_experts)` where each row sums to 1. We now want to map this to a per-expert assignments tensor `expert_assignments` of shape `(num_experts, expert_capacity)`, so that `expert_assignment[i, :]` contains all token ids assigned to expert `i`.

As the procedure might need to cut-off tokens when an expert is over-assigned, we will introduce an additional parameter `capacity_factor` which allows us to increase the per-expert capacity.

Let's now write several helper functions in order to tackle this.

#### Generate assignment matrix

Based on previously calculated scores, assign the `top-1` expert to each token and create a binary mask indicating the token-expert assignments:

In [ ]:
def get_expert_mask(scores: torch.Tensor) -> torch.Tensor:
    """Return expert_mask[i, j] = 1 if token i assigned to expert j, 0 otherwise"""
    token_assignments = torch.argmax(scores, dim=1)
    return F.one_hot(token_assignments, scores.shape[1])


# 4 tokens, 4 experts
scores = torch.tensor(
    [
        [0.1, 0.2, 0.5, 0.2],
        [0, 0, 0.9, 0.1],
        [0.4, 0.3, 0.15, 0.15],
        [0.2, 0.2, 0.2, 0.4],
    ]
)

expert_mask = get_expert_mask(scores)
print("Assignment matrix based on scores:")
print(expert_mask)

#### Assign tokens to experts

Since there are no guarantees the previously generated assignment matrix is balanced (i.e. that all *columns* sum to `expert_capacity`), we will now write a function that will ensure this property by only assigning the first `expert_capacity` tokens to an expert and ignoring the rest, and similarly, assigning the "dummy" token id `batch_size` to fill the under-assigned experts (note that the actual token ids span from `0` to `batch_size - 1`).

In [ ]:
def mask_to_expert_assignments(
    expert_mask: torch.Tensor, capacity: int
) -> torch.Tensor:
    """
    Convert token to expert assignment matrix (expert mask) to expert assignments
    of shape (num_experts, expert_capacity), discarding tokens for oversubscribed
    experts and padding undersubscribed experts
    """
    expert_mask_cumsum = torch.cumsum(expert_mask, dim=0)
    capacity_range = torch.arange(
        capacity, dtype=expert_mask.dtype, device=expert_mask.device
    )
    le_capacity = expert_mask_cumsum[:, :, None] <= capacity_range
    expert_assignments = le_capacity.sum(dim=0)
    return expert_assignments


print("Final expert assignments:")
print(mask_to_expert_assignments(expert_mask, capacity=1))

In this example with four tokens and four experts, each expert has the capacity of one token. As no tokens are assigned to expert 1, it gets the dummy token (token 4). Further, since both tokens 0 and 1 were assigned to expert 2, the assignment proceeds in order and token 0 is assigned to the expert, while token 1 remains unassigned. Tokens 2 and 3 are assigned to their prefered experts as there is no competition. The reader is encouraged to see what happens when the expert capacity is increased.

#### Load balancing

One issue with the current procedure is that it creates unbalanced token-expert assignments throughout training. If early in the training an expert receives more tokens, over time it will preferentially receive more tokens due to it being more trained. This can quickly lead to assignments collapsing to only a small subset of the experts, leading to many tokens being cut-off and the degradation in performance. In order to tackle this, we introduce a load balancing auxiliary loss that penalises unbalanced assignments.

We calculate this loss by averaging the per-token expert scores *before assignment* over the batch dimension (`density_proxy`), as well as calculating the *actual* proportions of tokens received by each expert by averaging `expert_mask` over the batch dimension (`density`). The final loss is then defined as the dot product between the two vectors. In a perfectly balanced situation all experts receive `1 / num_experts` proportion of the tokens, in which case `density[i] = 1 / num_experts` and `density_proxy[i] = 1 / num_experts` for each expert `i`. Therefore, we multiply the dot product with `num_experts` in order to normalise the balancing loss to `1.0` when all experts are balanced.

In [ ]:
def get_balancing_loss(scores: torch.Tensor, expert_mask: torch.Tensor = None):
    if expert_mask is None:
        expert_mask = get_expert_mask(scores)
    density = expert_mask.float().mean(dim=0)
    density_proxy = scores.mean(dim=0)
    return torch.dot(density, density_proxy) * density.shape[0]


scores_balanced = torch.tensor(
    [
        [0.7, 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, 0.7],
        [0.1, 0.7, 0.1, 0.1],
        [0.1, 0.1, 0.7, 0.1],
    ]
)
scores_unbalanced = torch.tensor(
    [
        [0.7, 0.1, 0.1, 0.1],
        [0.7, 0.1, 0.1, 0.1],
        [0.1, 0.7, 0.1, 0.1],
        [0.1, 0.1, 0.7, 0.1],
    ]
)

print("Balancing loss of a balanced assignment:")
print(get_balancing_loss(scores_balanced))
print("Balancing loss of an unbalanced assignment:")
print(get_balancing_loss(scores_unbalanced))

#### Putting it together

We now have everything we need to construct a Switch router that will take a batch of tokens and output per-expert token assignments. We'll make sure we shuffle the tokens before the procedures outlined above to ensure a fair cut-off of tokens (otherwise tokens appearing early in sequence would have an advantage!).

##### Shuffle tokens

In [ ]:
def shuffle_tokens(
    tokens: torch.Tensor, shuffle_idxs: torch.Tensor = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Shuffle tokens before assigning them to experts"""
    if shuffle_idxs is None:
        shuffle_idxs = torch.randperm(tokens.shape[0], device=tokens.device)
    return tokens[shuffle_idxs], shuffle_idxs


tokens = torch.arange(10) * 10
print("Original tokens:")
print(tokens)
print("Shuffled tokens:")
print(shuffle_tokens(tokens))

##### Full switch router

The final output of our router function should contain `expert_assignments` of shape `(num_experts, expert_capacity)`, corresponding token-expert scores `expert_gates` of the same shape, and the `balancing_loss` which we will use during training to ensure balanced token assignments across experts.

We define two different versions of the switch router, depending if the model is run in `training` or in `inference` mode:
* `training`: Expert capacity is calculated as `(batch_size / num_experts) * capacity_factor` and we keep track of `balancing_loss` in order to train the router.
* `inference`: We set the expert capacity to the maximum number of tokens assigned to an expert, thus ensuring that no tokens are dropped, while disregarding `balancing_loss`.

In [ ]:
@dataclass
class RouterOutput:
    expert_assignments: torch.Tensor
    expert_gates: torch.Tensor
    balancing_loss: Optional[torch.Tensor]

In [ ]:
def get_expert_gates(
    scores: torch.Tensor, expert_assignments: torch.Tensor
) -> torch.Tensor:
    pad_token = torch.zeros(
        (1, scores.shape[1]), dtype=scores.dtype, device=scores.device
    )
    scores_padded = torch.cat((scores, pad_token), 0)
    return torch.gather(scores_padded.transpose(0, 1), 1, expert_assignments)


# Test get_expert_gates
scores = torch.tensor(
    [
        [0.1, 0.2, 0.5, 0.2],
        [0, 0, 0.9, 0.1],
        [0.4, 0.3, 0.15, 0.15],
        [0.2, 0.2, 0.2, 0.4],
    ]
)

expert_assignments = torch.tensor([[0], [2], [1], [4]])
print("Expert assignments:\n", expert_assignments)
print("Corresponding expert gates:\n", get_expert_gates(scores, expert_assignments))

In [ ]:
def switch_router(
    scores: torch.Tensor, capacity_factor: float = 1.0, training: bool = True
) -> RouterOutput:
    expert_mask = get_expert_mask(scores)

    # At inference increase capacity so no tokens are dropped
    if not training:
        capacity = expert_mask.sum(dim=0).max()
        expert_assignments = mask_to_expert_assignments(expert_mask, capacity)
        expert_gates = get_expert_gates(scores, expert_assignments)
        return RouterOutput(expert_assignments, expert_gates, None)

    balancing_loss = get_balancing_loss(scores, expert_mask)
    expert_mask_shuffled, shuffle_idxs = shuffle_tokens(expert_mask)
    batch_size, num_experts = scores.shape
    capacity = int(batch_size * capacity_factor / num_experts)
    expert_assignments_shuffled = mask_to_expert_assignments(
        expert_mask_shuffled, capacity
    )
    shuffle_idxs_with_pad = torch.cat(
        (
            shuffle_idxs,
            torch.tensor(
                [scores.shape[0]], dtype=shuffle_idxs.dtype, device=shuffle_idxs.device
            ),
        )
    )
    expert_assignments = shuffle_idxs_with_pad[expert_assignments_shuffled]
    expert_gates = get_expert_gates(scores, expert_assignments)
    return RouterOutput(expert_assignments, expert_gates, balancing_loss)

In [ ]:
# Test switch router (training)
scores = torch.tensor(
    [
        [0.1, 0.2, 0.5, 0.2],
        [0.0, 0.0, 0.9, 0.1],
        [0.4, 0.3, 0.15, 0.15],
        [0.2, 0.2, 0.2, 0.4],
    ]
)

r = switch_router(scores, capacity_factor=1.0)

print("Expert assignments:\n", r.expert_assignments)
print("Expert gates:\n", r.expert_gates)
print("Balancing loss:\n", r.balancing_loss)

In [ ]:
# Test switch router (inference)
scores = torch.tensor(
    [
        [0.7, 0.1, 0.1, 0.1],
        [0.7, 0.1, 0.1, 0.1],
        [0.7, 0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2, 0.4],
    ]
)

r = switch_router(scores, training=False)

print("Expert assignments:\n", r.expert_assignments)
print("Expert gates:\n", r.expert_gates)
print("Balancing loss:\n", r.balancing_loss)

### Token routing

Having calculated the token-expert assignments using the `switch_router` function, we now need to route the tokens to their allocated experts. We consider the general case where the experts are distributed equally across our replicas, so that each replica holds `num_experts_per_replica` experts, and the total number of experts is therefore `num_experts = num_replicas * num_experts_per_replica`.

When using a single replica (either CPU or a single IPU), all experts live on the device and token routing simply gathers the appropriate tokens for each expert. When using multiple replicas however, each replica will hold a different subset of experts and we need to do cross-device communication in order to send the tokens to the correct device.

#### All-to-all

When using `num_replicas > 1`, each replica will send a subset of its token batch to each other replica. For this we will use the `all_to_all` op from the `poptorch_experimental_addons` library.

In [ ]:
from poptorch_experimental_addons.collectives import all_to_all_single_cross_replica


def all_to_all(x: torch.Tensor) -> torch.Tensor:
    if poptorch.isRunningOnIpu():
        return all_to_all_single_cross_replica(x, x.shape[0])
    else:
        assert x.shape[0] == 1, "num_replicas needs to be 1 when running on CPU"
        return x

The following example showcases `all_to_all` op: we start from the input `x` that is distributed across the `num_replicas` replicas, such that `x[i]` is the tensor on replica `i`. On each replica, the tensor `x[i]` needs to have the shape `(num_replicas, *)` before applying `all_to_all`, so that `x[i][j]` represents the data that replica `i` will send to replica `j`. After the all-to-all operation each replica `i` will have the tensor `y[i]`, such that `y[i][j] = x[j][i]`.

Note that by default when we send an input `x` with shape `(batch_size, *)` to a `poptorch_model` it will be distributed across the replicas so that each replica receives an independent slice of the input that will have the shape `(batch_size // num_replicas, *)`, which is why we do an additional reshape to have the input in this form. Similarly, the output will then have the shape `(batch_size, *)` which we reshape to `(num_replicas, batch_size // num_replicas, *)` in order to separate the outputs coming from each replica.

In [ ]:
def test_all_to_all():
    num_replicas = 4

    x = np.array(
        [
            np.arange(1, num_replicas + 1, dtype=np.int32)[:, np.newaxis] * 10**i
            for i in range(num_replicas)
        ]
    )

    for i, x_i in enumerate(x):
        print(f"Input on replica {i}:")
        print(x_i)

    x = x.reshape(x.shape[0] * x.shape[1], *x.shape[2:])

    class AllToAll(nn.Module):
        def forward(self, x: torch.Tensor):
            return all_to_all(x)

    model = AllToAll()
    opts = poptorch.Options()
    opts.replicationFactor(num_replicas)
    poptorch_model = poptorch.inferenceModel(model, opts)
    y = poptorch_model(torch.from_numpy(x))

    y = y.reshape((num_replicas, num_replicas, -1)).numpy()

    for i, y_i in enumerate(y):
        print(f"Output on replica {i}:")
        print(y_i)


# Uncomment to test all-to-all on 4 IPUs
# test_all_to_all()

#### Routing tokens to experts

Now that we have the `all_to_all` communication, we can route the tokens to the appropriate experts based on the previously calculated `expert_assignments`. We need to ensure that the tokens have the correct shape before the `all_to_all` (`num_replicas, num_experts_per_replica, expert_capacity, hidden_size`), as well as add the padding token for the under-assigned experts.

In [ ]:
def route_to_replica(
    x: torch.Tensor, expert_assignments: torch.Tensor, num_replicas: int
) -> torch.Tensor:
    assert len(x.shape) == 2

    pad_token = torch.zeros((1, x.shape[1]), dtype=x.dtype, device=x.device)
    x_padded = torch.cat((x, pad_token), 0)
    x_gathered = x_padded[
        expert_assignments.reshape(
            num_replicas,
            expert_assignments.shape[0] // num_replicas,
            *expert_assignments.shape[1:]
        ),
        :,
    ]
    return all_to_all(x_gathered)


# Test route
tokens = (torch.arange(8) * 2).reshape(-1, 1)
print("Tokens:\n", tokens)
e_assignments = torch.tensor([[1, 3], [0, 7], [5, 6], [2, 4]])
print("Expert assignments:\n", e_assignments)
out = route_to_replica(tokens, e_assignments, num_replicas=1)
out_expected = torch.tensor([[[2], [6]], [[0], [14]], [[10], [12]], [[4], [8]]])[
    None, :, :, :
]

print("Routing output:\n", out)

#### Rearrange expert inputs

After the `all_to_all`, each replica has a different set of inputs of the shape `x_routed = (num_replicas, num_experts_per_replica, expert_capacity, hidden_size)`. To support multiple experts per replica, within each replica we will run the experts concurrently using block-diagonal matrix multiplication for which we need to re-arrange the inputs to `x_allocated = (num_experts_per_replica, expert_capacity * num_replicas, hidden_size)`. We need to be careful here when reshaping in order to ensure that `x_allocated[i]` collects all tokens sent to expert `i` of the replica, i.e. `x_allocated[i] = x[:][i]`. For this, we need two functions: one to re-arrange expert inputs after `all_to_all`, and one to re-arrange the inputs back to the original form to be sent back after expert processing.

In [ ]:
def route_to_block(x: torch.Tensor) -> torch.Tensor:
    assert len(x.shape) == 4
    return x.swapaxes(0, 1).reshape(x.shape[1], -1, x.shape[-1])


def unroute_from_block(x: torch.Tensor, num_replicas: int) -> torch.Tensor:
    assert len(x.shape) == 3
    return x.reshape(x.shape[0], num_replicas, -1, x.shape[-1]).swapaxes(0, 1)


# Test allocate-deallocate replica experts
x = torch.tensor([[[[0]], [[1]]], [[[2]], [[3]]]])
print("Input before rearranging:\n", x)
print(
    "Input shape is (num_replicas, num_experts_per_replica, expert_capacity, hidden_size):\n",
    x.shape,
)
y = route_to_block(x)
print(
    "Rearranging to (num_experts_per_replica, num_replicas * expert_capacity, hidden_size):"
)
y_expected = np.array([[[0], [2]], [[1], [3]]])
print(y)
print("Shape of the output:\n", y.shape)
_x = unroute_from_block(y, num_replicas=2)
print("Inverting the re-arrangement:\n", _x)

#### Routing back

After the tokens have passed through the expert layers, they need to be rearranged and routed back to their original devices (i.e. the inverse operation of `route`). One addition here is that we need to multiply the expert outputs by the appropriate token-expert score so that we introduce the training signal for the router.

In [ ]:
def unroute_from_replica(
    x: torch.Tensor,
    expert_assignments: torch.Tensor,
    batch_size: int,
    expert_gates: Optional[torch.Tensor] = None,
) -> torch.Tensor:
    assert len(x.shape) == 4

    hidden_size = x.shape[-1]
    x_unrouted = all_to_all(
        x
    )  # (num_replicas, num_experts_per_replica, expert_capacity, hidden_size)
    if expert_gates is not None:
        x_unrouted *= expert_gates.reshape(*x_unrouted.shape[:-1], 1)
    x_flat = x_unrouted.reshape(-1, hidden_size)
    expert_assignments_flat = expert_assignments.reshape(-1, 1)
    y = torch.zeros((batch_size + 1, hidden_size), dtype=x.dtype, device=x.device)
    return torch.scatter_reduce(
        y, 0, expert_assignments_flat.expand(-1, hidden_size), x_flat, "sum"
    )[:-1]


# Test unroute without expert gates
x = torch.tensor(
    [[[2], [6]], [[0], [14]], [[10], [12]], [[4], [8]]], dtype=torch.float32
)[None, :, :, :]
print("Tokens to be routed back:\n", x)
e_assignments = torch.tensor([[1, 3], [0, 3], [5, 6], [2, 1]])
print("Based on expert assignments:\n", e_assignments)
y = unroute_from_replica(x, e_assignments, batch_size=np.prod(x.shape[:-1]))
y_expected = torch.tensor([0, 2 + 8, 4, 6 + 14, 0, 10, 12, 0]).reshape(-1, 1)
print("Tokens after unroute (no gating):\n", y)

# Test unroute with expert gates
x = torch.tensor(
    [[[2], [6]], [[0], [14]], [[10], [12]], [[4], [8]]], dtype=torch.float32
)[None, :, :, :]
e_assignments = torch.tensor([[1, 3], [0, 3], [5, 6], [2, 1]])
expert_gates = torch.tensor([[1, 1], [1, 1], [0.3, 1], [1, 0.5]])
print("\nSame expert assignments but with corresponding expert gates:\n", expert_gates)
y = unroute_from_replica(
    x, e_assignments, batch_size=np.prod(x.shape[:-1]), expert_gates=expert_gates
)
y_expected = torch.tensor([0, 2 + 0.5 * 8, 4, 6 + 14, 0, 0.3 * 10, 12, 0]).reshape(
    -1, 1
)
print("Tokens after unrouted (with gating):\n", y)

### Creating a MoE layer

We can now use the routing functions we defined to create expert-parallel layers. Let's create `ExpertFFN` - an expert-parallel version of our previous `FFN` layer. For each weight of the original layer of shape `w_shape` we now initialise an expert-parallel version instead, with shape `(num_replicas, num_experts_per_replica, *w_shape)` - in case of the `FFN` layer these are the weights `w1` and `w2`.

Note that each replica will only receive its own shard of this weight, so within the forward method of our class the shape will actually be `num_experts_per_replica, *w_shape` without the leading dimension. We need to tell `poptorch` during model definition which weights will be sharded in this way, as by default same weight values are *replicated* for each replica. We get rid of the leading `num_replicas` dimension in the case when `num_replicas == 1`.

We can now conduct our matrix multiplications as *batched* matmuls, as our inputs will have the same leading dimension `num_experts_per_replica` after the routing.

In [ ]:
class ExpertFFN(nn.Module):
    """Transformer-style FFN layer with expert-parallelism"""

    def __init__(
        self,
        hidden_size: int,
        expand_factor: int,
        dtype: torch.dtype,
        num_replicas: int,
        num_experts_per_replica: int,
        capacity_factor: float,
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.expand_factor = expand_factor
        self.dtype = dtype
        self.num_replicas = num_replicas
        self.num_experts_per_replica = num_experts_per_replica
        self.capacity_factor = capacity_factor
        self.scorer = Scorer(hidden_size, num_replicas * num_experts_per_replica, dtype)
        self.balancing_loss = None

        # Create expert params
        if num_replicas > 1:
            e_shape = (num_replicas, num_experts_per_replica)
        else:
            e_shape = (num_experts_per_replica,)

        self.w1 = create_weight_tensor(
            (
                *e_shape,
                hidden_size,
                hidden_size * expand_factor,
            ),
            dtype,
        )
        self.w2 = create_weight_tensor(
            (
                *e_shape,
                hidden_size * expand_factor,
                hidden_size,
            ),
            dtype,
        )

        # Keep track of router decisions
        self.expert_ids = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Make sure inputs are of shape (batch_size, hidden_size)
        x_shape = x.shape
        x = x.reshape(-1, x_shape[-1])
        batch_size = x.shape[0]

        # Assign tokens to experts
        probs = self.scorer(x)
        self.expert_ids = probs.argmax(dim=1)
        r = switch_router(probs, self.capacity_factor, self.training)
        self.balancing_loss = r.balancing_loss

        # Route + process + unroute
        x_routed = route_to_replica(x, r.expert_assignments, self.num_replicas)
        x_allocated = route_to_block(x_routed)
        y = F.relu(x_allocated @ self.w1) @ self.w2  # batched matmul
        y_deallocated = unroute_from_block(y, self.num_replicas)
        return unroute_from_replica(
            y_deallocated, r.expert_assignments, batch_size, r.expert_gates
        ).reshape(x_shape)

Let's also create a function that will take a regular dense `FFN` and return an expert-parallel `ExpertFFN` that we can later use to turn dense layers into MoE layers.

In [ ]:
def moe_ffn_like(
    ffn: FFN, num_replicas: int, num_experts_per_replica: int, capacity_factor: float
):
    return ExpertFFN(
        hidden_size=ffn.hidden_size,
        expand_factor=ffn.expand_factor,
        dtype=ffn.dtype,
        num_replicas=num_replicas,
        num_experts_per_replica=num_experts_per_replica,
        capacity_factor=capacity_factor,
    )

In [ ]:
# Test expert ffn
ffn = FFN(hidden_size=1, expand_factor=4, dtype=torch.float32)
e_ffn = moe_ffn_like(
    ffn, num_replicas=1, num_experts_per_replica=5, capacity_factor=1.0
)


# Test shapes
assert e_ffn.w1.shape == (5, 1, 1 * 4)
assert e_ffn.w2.shape == (5, 1 * 4, 1)
assert e_ffn.scorer.w.shape == (1, 5)
assert e_ffn.scorer.b.shape == (5,)
x = torch.randn(15, 1)
y = e_ffn(x)
assert y.shape == x.shape

## MoE character model

Let's now update our original character language model by replacing some of the `FFN` layers with our new `ExpertFFN` layers! A common approach to "sparsifying" a dense model is to take every other layer and make it MoE - so let's do that.

In addition to exchanging some `FFN` layers with `ExpertFFN` layers, we need to ensure that the expert balancing loss is included in the final loss calculation.

In [ ]:
class MoeCharModel(CharModel):
    def __init__(
        self,
        vocab_size: int,
        hidden_size: int,
        expand_factor: int,
        kernel_size: int,
        num_groups: int,
        num_layers: int,
        dtype: torch.dtype,
        num_replicas: int,
        num_experts_per_replica: int,
        capacity_factor: float,
        moe_layers: Tuple[int, ...],
        aux_loss_weight: float,
    ):
        super().__init__(
            vocab_size=vocab_size,
            hidden_size=hidden_size,
            expand_factor=expand_factor,
            kernel_size=kernel_size,
            num_groups=num_groups,
            num_layers=num_layers,
            dtype=dtype,
        )
        self.moe_layers = moe_layers
        self.aux_loss_weight = aux_loss_weight

        # Convert layers to MoE
        for i in moe_layers:
            self.core_layers[i].ffn = moe_ffn_like(
                self.core_layers[i].ffn,
                num_replicas,
                num_experts_per_replica,
                capacity_factor,
            )

    def forward(
        self, x: torch.Tensor, mask: torch.Tensor = None, y: torch.Tensor = None
    ) -> torch.Tensor:
        out = super().forward(x, mask, y)
        if self.training:
            logits, cross_entropy_loss = out
            aux_losses = {
                i: self.core_layers[i].ffn.balancing_loss
                for i in self.moe_layers
                if self.core_layers[i].ffn.balancing_loss is not None
            }
            aux_loss = sum(aux_losses.values())
            loss = cross_entropy_loss + self.aux_loss_weight * aux_loss
            return {
                "logits": logits,
                "cross entropy loss": cross_entropy_loss,
                "aux losses": aux_losses,
            }, poptorch.identity_loss(loss, reduction="none")
        else:
            return out

#### Training MoE model (*can be skipped*)

Finally, we can train our new model! We need to ensure that our expert parameters are properly sharded across the replicas - we will use this by setting the appropriate communication groups for the parameters of `ExpertFFN` layers, but making sure that the `Scorer` parameters are kept replicated across the replicas (as the router should be shared across each device).

In [ ]:
# # Load the training dataset

# vocab_path = dataset_dir / "wikitext103_raw/vocab.json"
# train_path = dataset_dir / "wikitext103_raw/train.txt"

# train_ds = Dataset.from_path(vocab_path, train_path)

In [ ]:
# # Set random seed
# seed = None
# if seed is None:
#     seed = np.random.SeedSequence().generate_state(1)[0]

# # Model hyperparameters
# num_replicas = 4
# num_experts_per_replica = 4
# model = MoeCharModel(
#     vocab_size=len(ds.vocab),
#     hidden_size=128,
#     expand_factor=4,
#     kernel_size=7,
#     num_groups=8,
#     num_layers=8,
#     dtype=torch.float32,
#     num_replicas=num_replicas,
#     num_experts_per_replica=num_experts_per_replica,
#     capacity_factor=1.0,
#     moe_layers=(0, 2, 4, 6),
#     aux_loss_weight=0.01,
# )

# # Batch settings
# batch_size = 32
# sequence_length = 256
# overlap_length = 32
# batch_seed = 100


# # Change this to switch between IPU and CPU execution
# run_on_ipu = True

# # Training parameters
# lr = 2e-3
# lr_decay = 2**-16
# num_steps = 2**15


# # Set the optimizer
# if run_on_ipu:
#     opts = poptorch.Options()
#     opts.replicationFactor(num_replicas)
#     optimizer = poptorch.optim.AdamW(model.parameters(), lr=lr)
#     poptorch_model = poptorch.trainingModel(model, opts, optimizer)

#     # Shard MoE params across replicas
#     if num_replicas > 1:
#         for i in model.moe_layers:
#             for p_name, _ in model.core_layers[i].ffn.named_parameters():
#                 # Router params should stay replicated
#                 if "scorer" not in p_name:
#                     poptorch_model.per_replica_params[f"core_layers.{i}.ffn." + p_name] = (
#                         poptorch.enums.CommGroupType.NoGrouping,
#                         0,
#                         poptorch.enums.VariableRetrievalMode.OnePerGroup,
#                     )


# else:
#     optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


# def mean_across_replicas(out: dict, num_replicas: int) -> dict:
#     """Average output values across replicas"""
#     if num_replicas == 1:
#         return out

#     return {
#         key: (
#             torch.mean(
#                 val.reshape(
#                     (num_replicas, val.shape[0] // num_replicas, *val.shape[1:])
#                 ),
#                 dim=0,
#                 dtype=torch.float32,
#             )
#             if type(val) is not dict
#             else mean_across_replicas(val, num_replicas)
#         )
#         for key, val in out.items()
#     }


# # Training loop
# model.train()
# lr_decay_factor = 2**-lr_decay
# for step, batch in zip(
#     range(num_steps),
#     train_ds.batch(batch_size, sequence_length, overlap_length, batch_seed),
# ):
#     # Apply a step
#     if run_on_ipu:
#         out, _ = poptorch_model(batch["x"].int(), batch["mask"].int(), batch["y"].int())
#         out = mean_across_replicas(out, num_replicas)
#     else:
#         optimizer.zero_grad()
#         out, loss = model(batch["x"], batch["mask"], batch["y"])
#         loss.backward()
#         optimizer.step()

#     # Print results
#     if step % 2**8 == 0:
#         print(f"Step {step}:")
#         print(f"Loss: {out['cross entropy loss']}")
#         print(f"Expert balancing losses: {out['aux losses']}")

#     # Learning rate schedule
#     lr *= lr_decay_factor
#     optimizer.param_groups[0]["lr"] = lr
#     if run_on_ipu:
#         poptorch_model.setOptimizer(optimizer)

#### Evaluating models

We can now test how our models perform and observe the effect of increasing the number of experts on the model performance. For this, we can either use models trained in the notebook, or one of the pre-trained models provided. We include pre-trained MoE models with 4, 8, and 16 experts.

In [ ]:
def evaluate_model(model: CharModel, ds: Dataset) -> float:
    model.train(True)

    # Batch settings
    batch_size = 32
    sequence_length = 256
    overlap_length = 32
    batch_seed = None

    opts = poptorch.Options()
    num_replicas = 4
    opts.replicationFactor(num_replicas)
    optimizer = poptorch.optim.SGD(model.parameters(), lr=0)
    poptorch_model = poptorch.trainingModel(model, opts, optimizer)
    if num_replicas > 1 and isinstance(model, MoeCharModel):
        for i in model.moe_layers:
            for p_name, _ in model.core_layers[i].ffn.named_parameters():
                if "scorer" not in p_name:
                    poptorch_model.per_replica_params[
                        f"core_layers.{i}.ffn." + p_name
                    ] = (
                        poptorch.enums.CommGroupType.NoGrouping,
                        0,
                        poptorch.enums.VariableRetrievalMode.OnePerGroup,
                    )

    losses = []
    for batch in ds.batch(batch_size, sequence_length, overlap_length, batch_seed):
        out, loss = poptorch_model(
            batch["x"].int(), batch["mask"].int(), batch["y"].int()
        )
        if isinstance(model, MoeCharModel):
            loss = out["cross entropy loss"]
        losses.append(torch.mean(loss))
    return torch.mean(loss)

In [ ]:
# Load the test dataset
vocab_path = dataset_dir / "wikitext103_raw/vocab.json"
test_path = dataset_dir / "wikitext103_raw/test.txt"
test_ds = Dataset.from_path(vocab_path, test_path)

In [ ]:
# # Evaluate dense model

# model = CharModel(
#     vocab_size=len(ds.vocab),
#     hidden_size=128,
#     expand_factor=4,
#     kernel_size=7,
#     num_groups=8,
#     num_layers=8,
#     dtype=torch.float32,
# )
# model.load_state_dict(torch.load(checkpoint_dir / "dense.pt"))

# print(f"Baseline model loss on test set: {evaluate_model(model, test_ds).item():.3f}")

In [ ]:
# # Evaluate MoE models

# # Set number of experts (4, 8, or 16)
# num_experts = 4

# model = MoeCharModel(
#     vocab_size=len(ds.vocab),
#     hidden_size=128,
#     expand_factor=4,
#     kernel_size=7,
#     num_groups=8,
#     num_layers=8,
#     dtype=torch.float32,
#     num_replicas=4,
#     num_experts_per_replica=num_experts // 4,
#     capacity_factor=1.0,
#     moe_layers=(0, 2, 4, 6),
#     aux_loss_weight=0.01,
# )

# model.load_state_dict(torch.load(checkpoint_dir / f"moe-{num_experts}.pt"))

# print(
#     f"MoE model ({num_experts} experts) loss on test set: {evaluate_model(model, test_ds).item():.3f}"
# )

We should observe that increasing the number of experts indeed improved the performance of the model! Note that importantly, this improvement comes without a proportional increase in compute cost - mixture of expert scheme only adds the routing logic overhead, but the processing FLOPs per token remain fixed.

#### Running MoE inference

Let's see our MoE models in action! We will run the inference on the CPU, so we need to make sure that any expert weights of the shape `(num_replicas, num_experts_per_replica, ...)` are reshaped to `(num_replicas * num_experts_per_replica, ...)` as expected by the CPU model.

In [ ]:
def flatten_expert_weights(state_dict: dict, moe_layers: Tuple[int, ...]) -> dict:
    for i in moe_layers:
        w1 = state_dict[f"core_layers.{i}.ffn.w1"]
        w2 = state_dict[f"core_layers.{i}.ffn.w2"]
        state_dict[f"core_layers.{i}.ffn.w1"] = w1.reshape(-1, *w1.shape[2:])
        state_dict[f"core_layers.{i}.ffn.w2"] = w2.reshape(-1, *w2.shape[2:])
    return state_dict

We can also keep track of the router decisions by colouring each input character based on the expert it was routed to:

In [ ]:
def wrap_text(text: str, id: int) -> str:
    col_id = id + 1 if id < 15 else id + 20
    return "\033[38;5;232m" + f"\033[48;5;{col_id}m" + text + "\033[0;0m"

In [ ]:
# Set number of experts (can be 4, 8 or 16)
num_experts = 4

model_from_checkpoint = MoeCharModel(
    vocab_size=len(ds.vocab),
    hidden_size=128,
    expand_factor=4,
    kernel_size=7,
    num_groups=8,
    num_layers=8,
    dtype=torch.float32,
    num_replicas=1,
    num_experts_per_replica=num_experts,
    capacity_factor=1.0,
    moe_layers=(0, 2, 4, 6),
    aux_loss_weight=0.01,
)
state_dict = flatten_expert_weights(
    torch.load(checkpoint_dir / f"moe-{num_experts}.pt"), moe_layers=(0, 2, 4, 6)
)
model_from_checkpoint.load_state_dict(state_dict)

in_text = "This is a sentence to see how characters are routed to experts.\nSome numbers: 1, 2, 3, 4, 5, 6, 7, 8, 9.\n "
output = sample_text(
    model_from_checkpoint,
    in_text,
    num_characters=1,
    temperature=0.3,
)
for i in (0, 2, 4, 6):
    print(f"Layer {i}:")
    expert_ids = model_from_checkpoint.core_layers[i].ffn.expert_ids.numpy()
    colored_text = ""
    for c, id in zip(output, expert_ids):
        colored_text += wrap_text(c, id)
    print(colored_text)

## Conclusion

This notebook provided a tutorial on how to define a model that uses a mixture of experts execution scheme to increase the model capacity without a proportional increase in computational cost. Although the notebook covered defining a character-level language model, the techniques described here can be easily applied to making arbitrary expert-parallel layers. Moreover, by simply exchanging the FFN layers with `ExpertFFN` layers defined here and adding the expert-balancing loss to our final loss calculation, we can adapt arbitrary transformer-like models to utilise mixture of experts and run on both IPUs and CPU.